# MDP Para el juego Black Jack (21)

$ MDP = <S,A, \rho ,S_f,r > $

$ S = (d,sum,n)$ 

$ A = \{ fin, otra \} $

$ \rho = ...$ 

$ S_f =  \{ (0,0,0,1), (0,0,0,-1) \} $

$ r = $ 1 si gana, -1 si pierde y 0 en otro caso


## Definiciones principales del Modelo

In [1]:
abstract type MDP end
abstract type Estado end
abstract type Accion end

In [2]:
struct BJ_s <:Estado
    d #dealer
    sum #suma del jugador
    n #numero de cartas
    as #indica si se puede sumar 10 a sum o no. (1,0)

    function BJ_s(d::Int,sum::Int,n::Int,as::Int)
        if (d > 10 || sum>22 || n>4 || as>1 )
            error("estado invalido")
        else
            return new(d,sum,n,as)
        end
    end
end


In [3]:
BJ_s(5,0,0,-1)

BJ_s(5, 0, 0, -1)

In [4]:
struct BJ_a <:Accion
    a
    function BJ_a(a)
        if (a != :fin && a!= :otra)
            error("accion invalida")
        else
            new(a)
        end
    end
end

In [5]:
BJ_a(:otra)

BJ_a(:otra)

## Funciones del Modelo

In [6]:
#terminal
function terminal(s::BJ_s)
    (s.d==s.sum==s.n==0) && (s.as==1 || s.as==-1) ? true : false
end


#acciones legales
function aLegales(s::BJ_s)
    if terminal(s)
        return nothing
    end
    
    acciones = []
    
    if s.sum<=21 && s.n<4
        push!(acciones,BJ_a(:otra))
    end
    
    push!(acciones,BJ_a(:fin))
    
    acciones
end
                        
#recompenza
function r(s::BJ_s)
    if s.d==s.sum==s.n==0 && s.as==1
        return 1
    elseif s.d==s.sum==s.n==0 && s.as==-1
        return -1
    end
    0
end


r (generic function with 1 method)

In [7]:
# Funciones auxiliares

"""
Calcula el numero de combinaciones de dos cartas cuya suma esta entre dos valores. (izq_ , der_)
"""

function comb2cartas(izq_,der_)
    total=0; i=1 
    izq=izq_-i
    der=der_-i
    
    while (der>0 && i<=11)
        y=0
        if izq<9 y+=min(9,der)-max(izq,0) end
        if der>=10 y+=4 end
        if der>=11 y+=1 end
        i==10 ? total+=4*min(y,13) : total+=min(y,13)
        i+=1
        izq-=1
        der-=1
    end
    #println("total de combinaciones ",total)
    total
end

"""
Calcula el numero de combinaciones de tres cartas cuya suma esta entre dos valores. (izq_ , der_)
"""

function comb3cartas(izq_,der_)
    total=0
    for i in 1:11
        total+=comb2cartas(izq_ - i, der_ -i )
    end
    total
end



In [8]:
#probabilidad de Transicion
function ρ(s::BJ_s,a::BJ_a,s2::BJ_s)
    
    if  terminal(s) 
        return 0
    end
    
    if a ∉ aLegales(s)
        return 0
    end
    
    if a.a == :fin
        if s2.d==s2.sum==s2.n==0 && s2.as==1 #si gana
            s.as==1 && s.sum+10< 22 ? sum=s.sum+10 : sum=s.sum
            if sum==21 return 1 end
            
            #println("sum= ",sum)
            x=0 #guardara todos los casos en los que puede perder el dealer.
            
            if sum >= s.d # si tiene mas que el dealer
                dif = sum-s.d
                #Formas en las que puede perder el dealer ...
                
                #con 1 carta (se queda con la primera solamente)
                x+=1
                
                #con 2 cartas
                if dif<10 
                    x+=dif-1
                    if s.d+11 >21 x+=1 end
                elseif dif==10 x+=12 
                else x+=13
                end
            
                #con 3 cartas
                x+=169 - comb2cartas(dif,21-s.d)
                #println("x= ",x)
                #con 4 cartas
                x+=2197 - comb3cartas(dif,21-s.d)
                #println("x= ",x)
                
                return x/2380
            else
            # si tiene menos que el dealer. La probabilidad de que el dealer se pase
            # ...
                x+=169 - comb2cartas(s.d,21)
                x+=2197 - comb3cartas(s.d,21)
                #println("x= ",x)
                
                return x/2380
            
            end
            
            
            
        elseif s2.d==s2.sum==s2.n==0 && s2.as==-1 #si pierde
            #println("entre...")
            return 1 - ρ(s,a,BJ_s(0,0,0,1))
        
        #else: pasar a otro estado que no sea ganar ni perder
        end
        
    elseif a.a== :otra
        if s2.d==s2.sum==s2.n==0 && s2.as==1 #si gana automaticamente. (llegar a las 4 cuartas y sum<21)
            if s.n==3
                dif=21-s.sum
                if dif<10 return (dif/13) #no me dejo dif<10 ? return dif/13:1
                else return 1
                end
            #else: cundo no tendra 4 cartas e inclusive puede tener 21. Solo que necesita una jugada 
            #extra donde decida finalizar
            end
        elseif s2.d==s2.sum==s2.n==0 && s2.as==-1 #si pierde automaticamente.(cuando se pase de 21)
            dif=22-s.sum
            if dif<10  return (10-dif+4)/13
            elseif dif==10 return 4/13
            #else: cuando la diferencia sea mas de lo que puede sumar una carta
            end
        elseif s2.d ==s.d && s2.n==s.n+1 #si aumenta en 1 el num de cartas y el dealer es el mismo
                if s2.sum-s.sum<10 && s2.sum-s.sum>1 #si anda entre 2 y 9 lo que aumenta la suma
                    return 0.0769 #1/13
                elseif s2.sum-s.sum==1 # si aumento en 1 la suma, solo puede ser un As
                    if s2.as==1 return 0.0769 end                    
                elseif s2.sum-s.sum==10 #si aumenta en 10 hay 4 posibilidades
                    return 0.3077
                end 
        end
    
    end
    
    0 #todos los else
        
end

ρ (generic function with 1 method)

In [9]:
s1 = BJ_s(10,20,3,0)

println(typeof(s1))

println(terminal(s1))

show(aLegales(s1))

println()

println(r(s1))

println(ρ(s1, BJ_a(:otra), BJ_s(5,14,3,1)))
println(ρ(s1, BJ_a(:otra), BJ_s(5,14,3,0)))
println(ρ(BJ_s(5,12,1,0), BJ_a(:otra), BJ_s(5,14,2,0)))
println(ρ(BJ_s(4,7,1,0) , BJ_a(:otra), BJ_s(0,0,0,1)))
println(ρ(BJ_s(7, 20, 2, 1) , BJ_a(:otra), BJ_s(0,0,0,-1)))

println(ρ(BJ_s(0,0,0,-1), BJ_a(:fin), BJ_s(0,0,0,-1)))

BJ_s
false
Any[BJ_a(:otra), BJ_a(:fin)]
0
0
0
0.0769
0
0.9230769230769231
0


## Generacion de estados

In [10]:
est=[BJ_s(1,1,1,0)]
pop!(est)

for k in 1:4 #num
    if k==1
        x=10 
    elseif k==2 
        x=20
    else
        x=21
    end
    for i in 1:10 #dealer
        for j in 1:x #sum
            for l in 0:1 #as
                if j>k
                    if l==1 && j<=(k-1)*10
                        push!(est,BJ_s(i,j,k,l))
                    elseif l==0
                        push!(est,BJ_s(i,j,k,l))
                    end
                end
            end
        end
    end
end

push!(est,BJ_s(0,0,0,1))
push!(est,BJ_s(0,0,0,-1))

est

1042-element Array{BJ_s,1}:
 BJ_s(1, 2, 1, 0)  
 BJ_s(1, 3, 1, 0)  
 BJ_s(1, 4, 1, 0)  
 BJ_s(1, 5, 1, 0)  
 BJ_s(1, 6, 1, 0)  
 BJ_s(1, 7, 1, 0)  
 BJ_s(1, 8, 1, 0)  
 BJ_s(1, 9, 1, 0)  
 BJ_s(1, 10, 1, 0) 
 BJ_s(2, 2, 1, 0)  
 BJ_s(2, 3, 1, 0)  
 BJ_s(2, 4, 1, 0)  
 BJ_s(2, 5, 1, 0)  
 ⋮                 
 BJ_s(10, 17, 4, 0)
 BJ_s(10, 17, 4, 1)
 BJ_s(10, 18, 4, 0)
 BJ_s(10, 18, 4, 1)
 BJ_s(10, 19, 4, 0)
 BJ_s(10, 19, 4, 1)
 BJ_s(10, 20, 4, 0)
 BJ_s(10, 20, 4, 1)
 BJ_s(10, 21, 4, 0)
 BJ_s(10, 21, 4, 1)
 BJ_s(0, 0, 0, 1)  
 BJ_s(0, 0, 0, -1) 

## Generacion de acciones

In [11]:
ac = [BJ_a(:fin), BJ_a(:otra)]
ac

2-element Array{BJ_a,1}:
 BJ_a(:fin) 
 BJ_a(:otra)

## Modelo MDP

In [12]:
struct BJ <: MDP
    estados
    acciones
    function BJ(estados::Array{BJ_s,1}, acciones::Array{BJ_a,1})
        new(estados,acciones)
    end
end

In [13]:
BlackJ = BJ(est,ac)



## Algoritmo de Programacion Dinamica

In [14]:
function policy_value(mdp::MDP,π::Dict, γ::Real)
    
    v = Dict(si => 0 for si in mdp.estados)
    
    flag=true
    
    while flag
        flag=false
        
        
        for s in keys(π)
            
            #temp=sum([ρ(s,π[s],sp)==0 ? r(s) : ρ(s,π[s],sp)*(r(s)+γ*v[sp]) for sp in keys(v)])
            temp=sum([ρ(s,π[s],sp)*(r(s)+γ*v[sp]) for sp in keys(v)])
            
            if temp!=v[s]
                flag=true
            end
            v[s]=temp
        end
    end
    
    v
end

policy_value (generic function with 1 method)

In [15]:
function choice(a::Array)
    n = length(a)
    idx = rand(1:n)
    return a[idx]
end

choice([1,5,8,9])

8

In [17]:
function policy_iteration(mdp::MDP,γ::Real)
    
    π = Dict(s => rand(aLegales(s)) for s in mdp.estados if !terminal(s))
             
    optima=false
    
   
    while !optima
        v = policy_value(mdp,π,γ)
                    
        optima=true

        for s in keys(π)
            for a in mdp.acciones
                #temp=sum( ρ(s,π[s],s2)==0 ? r(s) : ρ(s,a,s2)*(r(s)+γ*v[s2]) for s2 in keys(v))
                temp=sum( [ρ(s,a,s2)*(r(s)+γ*v[s2]) for s2 in keys(v)])
                if temp < v[s] # 
                    optima=false
                    π[s]=a
                end
            end
        end
    end
    
    π
end

policy_iteration (generic function with 1 method)

In [18]:
π_opt= policy_iteration(BlackJ,1)

π_opt

Dict{BJ_s,BJ_a} with 1040 entries:
  BJ_s(8, 16, 3, 1)  => BJ_a(:fin)
  BJ_s(3, 8, 3, 0)   => BJ_a(:otra)
  BJ_s(8, 8, 4, 0)   => BJ_a(:fin)
  BJ_s(9, 10, 4, 0)  => BJ_a(:fin)
  BJ_s(4, 7, 1, 0)   => BJ_a(:fin)
  BJ_s(7, 8, 3, 0)   => BJ_a(:otra)
  BJ_s(8, 15, 4, 0)  => BJ_a(:fin)
  BJ_s(3, 15, 3, 1)  => BJ_a(:otra)
  BJ_s(5, 18, 3, 0)  => BJ_a(:fin)
  BJ_s(7, 9, 4, 1)   => BJ_a(:fin)
  BJ_s(6, 10, 2, 0)  => BJ_a(:otra)
  BJ_s(6, 16, 3, 0)  => BJ_a(:otra)
  BJ_s(10, 8, 4, 0)  => BJ_a(:fin)
  BJ_s(2, 13, 2, 0)  => BJ_a(:fin)
  BJ_s(10, 13, 3, 1) => BJ_a(:otra)
  BJ_s(8, 6, 2, 0)   => BJ_a(:fin)
  BJ_s(10, 4, 3, 1)  => BJ_a(:fin)
  BJ_s(2, 14, 4, 1)  => BJ_a(:fin)
  BJ_s(7, 6, 1, 0)   => BJ_a(:fin)
  BJ_s(4, 12, 3, 0)  => BJ_a(:otra)
  BJ_s(2, 4, 2, 0)   => BJ_a(:fin)
  BJ_s(9, 21, 3, 0)  => BJ_a(:fin)
  BJ_s(1, 12, 4, 0)  => BJ_a(:fin)
  BJ_s(1, 12, 3, 1)  => BJ_a(:otra)
  BJ_s(7, 8, 1, 0)   => BJ_a(:otra)
  ⋮                  => ⋮

In [46]:
function iter_value(mdp::MDP,γ::Real)
    
    V = Dict(s => 0 for s in mdp.estados)
    Vp = Dict(s => 0 for s in mdp.estados)
    
             
    optima=false
    
   
    while !optima
        
        optima = true
        for s in keys(V)
            
            opc = []
            for a in mdp.acciones
                suma=0
                for s2 in mdp.estados
                    suma+= ρ(s,a,s2)==0 ? r(s) : ρ(s,a,s2)*(r(s)+γ*V[s2])
                end
                push!(opc,suma)
                
            end
            println(opc)
            println("antes ", typeof(opc))
            
            Vp[s]=maximum(opc)
            println("despues")
            
            """
            Vp[s]=maximum([sum([ ρ(s,a,s2)==0 ? r(s) : ρ(s,a,s2)*(r(s)+γ*V[s2]) for s2 in mdp.estados ]) 
                    for a in mdp.acciones])"""
            
            if Vp[s]>V[s] 
                optima=false
                V[s]=Vp[s]
            end
            
        end
        
    end
    

    s_legales=[s for s in mdp.estados if !terminal(s)]
    
    sx=rand(s_legales)
                            
    π_ = Dict(sx => rand(aLegales(sx)))
    
    for s in s_legales 
       
        π_[s]=findmax(Dict(a => sum([ρ(s,a,s2)*(r(s)+γ*V[s2]) for s2 in mdp.estados]))  for a in mdp.acciones)[2]
    end
    
    π_ 
    
end

iter_value (generic function with 1 method)

In [ ]:
π_opt= iter_value(BlackJ,1)

π_opt